In [1]:
# import tensorflow as tf
# x = tf.placeholder(tf.float32, [None, 3])
# print(x)

In [2]:
# x_data = [[1, 2, 3], [4, 5, 6]]
# print(x_data)

In [3]:
# w = tf.Variable(tf.random_normal([3, 2]))
# b = tf.Variable(tf.random_normal([2, 1]))
# y = tf.matmul(x, w) + b

In [4]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     print(sess.run(w))
#     print(sess.run(b))
#     print("y:", sess.run(y, feed_dict={x: x_data}))

In [5]:
# x_data = [1, 2, 3]
# y_data = [1, 2, 3]

# W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
# b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

# X = tf.placeholder(tf.float32, name="X")
# Y = tf.placeholder(tf.float32, name="Y")

# hypo = W * X + b

# cost = tf.reduce_mean(tf.square(hypo - Y))
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
# train_op = optimizer.minimize(cost)

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    
#     for step in range(30):
#         _, cost_val = sess.run([train_op, cost], feed_dict={X:x_data, Y:y_data})
#         print("STEP:{}, COST_VAL:{}, W:{}, b:{}".format(step, cost_val, sess.run(W), sess.run(b)))
# #         print(step, cost_val, sess.run(W), sess.run(b))
        
#     print("X:5, Y:", sess.run(hypo, feed_dict={X:5}))
#     print("X:3, Y:", sess.run(hypo, feed_dict={X:3}))

In [6]:
# import numpy as np
# x_ = np.array([
#     [0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]
# ])
# y_ = np.array([
#     [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1]
# ])

# X = tf.placeholder(tf.float32)
# Y = tf.placeholder(tf.float32)

# W = tf.Variable(tf.random_uniform([2, 3], -1.0, 1.0))
# b = tf.Variable(tf.zeros([3]))

# L = tf.add(tf.matmul(X, W), b)
# L = tf.nn.relu(L)

# model = tf.nn.softmax(L)

# cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
# train_op = optimizer.minimize(cost)

# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)
    
#     for step in range(200):
#         sess.run(train_op, feed_dict={X:x_, Y:y_})
#         if (step + 1) % 10 == 0:
#             print(step + 1, sess.run(cost, feed_dict={X:x_, Y:y_}))
    
#     prediction = tf.argmax(model, axis=1)
#     target = tf.argmax(Y, axis=1)
    
#     print("예측값:", sess.run(prediction, feed_dict={X:x_}))
#     print("실제값:", sess.run(target, feed_dict={Y:y_}))
    
#     is_correct = tf.equal(prediction, target)
#     accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
#     print("정확도: %2f" % sess.run(accuracy * 100, feed_dict={X:x_, Y:y_}))

In [7]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data/species.csv', delimiter=',', unpack=True, dtype='float32', encoding='utf-8-sig')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1.0, 1.0))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1.0, 1.0))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1.0, 1.0))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
for step in range(2):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    
    print('Step: %d, ' % sess.run(global_step), 
         'Cost: %.3f' % sess.run(cost, feed_dict={X:x_data, Y:y_data}))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X:x_data}))
print('실제값:', sess.run(target, feed_dict={Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model\dnn.ckpt-6
Step: 7,  Cost: 0.686
Step: 8,  Cost: 0.660
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
